In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
import os

In [4]:
!pwd
!ls -l ./

/data/datadrive01/prj/tlvis
total 2188
-rw-rw-r-- 1 loginVM_001 loginVM_001  323359 Sep 23 06:38 00_create_docker_image.html
-rw-rw-r-- 1 loginVM_001 loginVM_001 1910154 Sep 23 06:39 00_create_docker_image.ipynb
drwxrwxr-x 2 loginVM_001 loginVM_001    4096 Sep 22 07:18 docker


In [5]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/chestxray-no-aml-gpu"

Overwriting .env


In [6]:
%%writefile ./docker/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER keras
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow-gpu \
      tensorflow-tensorboard \
      matplotlib \
      numpy && \
    conda install \
      bcolz \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      requests \
      scikit-learn \
      six \
      tqdm && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    pip install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl \
      torchvision && \
    conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./docker/dockerfile


In [7]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':1.0.0'
docker_file_location = os.path.join(*(['.', 'docker', 'dockerfile']))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu:1.0.0'

'./docker/dockerfile'

In [8]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu:1.0.0 -f ./docker/dockerfile .
Sending build context to Docker daemon  4.152MB
Step 1/16 : FROM nvidia/cuda:9.0-cudnn7-devel
 ---> 288866c2c06c
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> e90df2548ef0
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget &&     rm -rf /var/lib/apt/lists/*
 ---> Running in e3b68ee28cff
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  Rele

Get:19 http://archive.ubuntu.com/ubuntu xenial/main amd64 libsm6 amd64 2:1.2.2-1 [15.8 kB]
Get:20 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxdamage1 amd64 1:1.1.4-2 [6946 B]
Get:21 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxfixes3 amd64 1:5.0.1-2 [11.1 kB]
Get:22 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxshmfence1 amd64 1.2-1 [5042 B]
Get:23 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxxf86vm1 amd64 1:1.1.4-1 [10.6 kB]
Get:24 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjbig0 amd64 2.1-3.1 [26.6 kB]
Get:25 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libapt-inst2.0 amd64 1.2.27 [55.4 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 apt-utils amd64 1.2.27 [196 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial/main amd64 libbsd0 amd64 0.8.2-1 [41.7 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libdrm-common all 2.4.91-2~16.04.1 [4764 B]
Get:29 http://archive.ubuntu.com/ubun

debconf: delaying package configuration, since apt-utils is not installed
Fetched 43.1 MB in 3s (11.6 MB/s)
Selecting previously unselected package libglib2.0-0:amd64.
(Reading database ... 11603 files and directories currently installed.)
Preparing to unpack .../libglib2.0-0_2.48.2-0ubuntu4.1_amd64.deb ...
Unpacking libglib2.0-0:amd64 (2.48.2-0ubuntu4.1) ...
Selecting previously unselected package libxau6:amd64.
Preparing to unpack .../libxau6_1%3a1.0.8-1_amd64.deb ...
Unpacking libxau6:amd64 (1:1.0.8-1) ...
Selecting previously unselected package libxdmcp6:amd64.
Preparing to unpack .../libxdmcp6_1%3a1.1.2-1.1_amd64.deb ...
Unpacking libxdmcp6:amd64 (1:1.1.2-1.1) ...
Selecting previously unselected package libxcb1:amd64.
Preparing to unpack .../libxcb1_1.11.1-1ubuntu1_amd64.deb ...
Unpacking libxcb1:amd64 (1.11.1-1ubuntu1) ...
Selecting previously unselected package libx11-data.
Preparing to unpack .../libx11-data_2%3a1.6.3-1ubuntu2.1_all.deb ...
Unpacking libx11-data (2:1.6.3-1ubunt

Selecting previously unselected package libthai-data.
Preparing to unpack .../libthai-data_0.1.24-2_all.deb ...
Unpacking libthai-data (0.1.24-2) ...
Selecting previously unselected package libdatrie1:amd64.
Preparing to unpack .../libdatrie1_0.2.10-2_amd64.deb ...
Unpacking libdatrie1:amd64 (0.2.10-2) ...
Selecting previously unselected package libthai0:amd64.
Preparing to unpack .../libthai0_0.1.24-2_amd64.deb ...
Unpacking libthai0:amd64 (0.1.24-2) ...
Selecting previously unselected package libpango-1.0-0:amd64.
Preparing to unpack .../libpango-1.0-0_1.38.1-1_amd64.deb ...
Unpacking libpango-1.0-0:amd64 (1.38.1-1) ...
Selecting previously unselected package libgraphite2-3:amd64.
Preparing to unpack .../libgraphite2-3_1.3.10-0ubuntu0.16.04.1_amd64.deb ...
Unpacking libgraphite2-3:amd64 (1.3.10-0ubuntu0.16.04.1) ...
Selecting previously unselected package libharfbuzz0b:amd64.
Preparing to unpack .../libharfbuzz0b_1.0.1-1ubuntu0.1_amd64.deb ...
Unpacking libharfbuzz0b:amd64 (1.0.1-1ub

Selecting previously unselected package openmpi-bin.
Preparing to unpack .../openmpi-bin_1.10.2-8ubuntu1_amd64.deb ...
Unpacking openmpi-bin (1.10.2-8ubuntu1) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Processing triggers for systemd (229-4ubuntu21.4) ...
Setting up libglib2.0-0:amd64 (2.48.2-0ubuntu4.1) ...
No schema files found: doing nothing.
Setting up libxau6:amd64 (1:1.0.8-1) ...
Setting up libxdmcp6:amd64 (1:1.1.2-1.1) ...
Setting up libxcb1:amd64 (1.11.1-1ubuntu1) ...
Setting up libx11-data (2:1.6.3-1ubuntu2.1) ...
Setting up libx11-6:amd64 (2:1.6.3-1ubuntu2.1) ...
Setting up libxext6:amd64 (2:1.3.3-1) ...
Setting up fonts-dejavu-core (2.35-1) ...
Setting up ucf (3.0036) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Setting up fontconfig-config (2.11.94-0ubuntu1.1) ...
Setting up libexpat1:amd64 (2.1.0-7ubuntu0.16.04.3) ...
Setting up libpng12-0:amd6

...

blas-1.0-mkl.t 100% |###############################| Time: 0:00:00   8.53 MB/s
conda-env-2.6. 100% |###############################| Time: 0:00:00   5.53 MB/s
intel-openmp-2 100% |###############################| Time: 0:00:00  15.65 MB/s
libgcc-ng-8.2. 100% |###############################| Time: 0:00:00  61.59 MB/s
libgfortran-ng 100% |###############################| Time: 0:00:00  60.34 MB/s
libstdcxx-ng-8 100% |###############################| Time: 0:00:00  62.35 MB/s
bzip2-1.0.6-h1 100% |###############################| Time: 0:00:00  60.11 MB/s
expat-2.2.6-he 100% |###############################| Time: 0:00:00  59.53 MB/s
freeglut-3.0.0 100% |###############################| Time: 0:00:00  60.80 MB/s
gmp-6.1.2-h6c8 100% |###############################| Time: 0:00:00  63.33 MB/s
graphite2-1.3. 100% |###############################| Time: 0:00:00  57.25 MB/s
icu-58.2-h9c2b 100% |###############################| Time: 0:00:00  62.78 MB/s
jpeg-9b-h024ee 100% |###################

pytables-3.4.4 100% |###############################| Time: 0:00:00  56.23 MB/s MB/s
scikit-learn-0 100% |###############################| Time: 0:00:00  58.61 MB/s
bcolz-1.2.1-py 100% |###############################| Time: 0:00:00  60.51 MB/s
Cloning into '/src'...
Obtaining file:///src
  Running setup.py bdist_wheel for pytest-pep8: started
  Running setup.py bdist_wheel for pytest-pep8: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/68/34/52/2a6e08e030cef8b3671898f34cecb12750cc4b96ebfe66fe67
  Running setup.py bdist_wheel for pytest-cache: started
  Running setup.py bdist_wheel for pytest-cache: finished with status 'done'
  Stored in directory: /home/keras/.cache/pip/wheels/e4/9e/28/59d0a23800e49808f17920c5922252c661966ca4a36db5ee38
Successfully built pytest-pep8 pytest-cache
  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-ecmacwzv
  Running setup.py bdist_wheel for Keras: started
  Runnin

  Running setup.py bdist_wheel for imgaug: started
  Running setup.py bdist_wheel for imgaug: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-zyai011e/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-yrfp4gtx
  Running setup.py bdist_wheel for keras-contrib: started
  Running setup.py bdist_wheel for keras-contrib: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-ag9rcoct/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
asn1crypto-0.22.0-py36_0.tar.bz2             148 KB
snappy-1.1.7-hbae5bb6_3.tar.bz2               38 KB
cloudpickle-0.5.5-py36_0.tar.bz2              26 KB
wheel-0.29.0-py36_0.tar.bz2                   88 KB
jpeg-9b-h024ee3a_2.tar.bz2   

pixman-0.34.0-hceecf20_3.tar.bz2             598 KB
harfbuzz-1.8.8-hffaf4a1_0.tar.bz2            863 KB
pexpect-4.6.0-py36_0.tar.bz2                  77 KB
tk-8.6.8-hbc83047_0.tar.bz2                  3.1 MB
libgfortran-ng-7.3.0-hdf63c60_0.tar.bz2      1.3 MB
bleach-2.1.4-py36_0.tar.bz2                   33 KB
pyzmq-17.1.2-py36h14c3975_0.tar.bz2          454 KB
jsonschema-2.6.0-py36_0.tar.bz2               62 KB
automat-0.7.0-py36_0.tar.bz2                  52 KB
cffi-1.10.0-py36_0.tar.bz2                   341 KB
notebook-5.6.0-py36_0.tar.bz2                7.4 MB
bzip2-1.0.6-h14c3975_5.tar.bz2               414 KB
ipykernel-4.9.0-py36_1.tar.bz2               146 KB
scikit-learn-0.19.2-py36h4989274_0.tar.bz2     5.2 MB
pyasn1-modules-0.2.2-py36_0.tar.bz2           86 KB
cytoolz-0.9.0.1-py36h14c3975_1.tar.bz2       419 KB
freetype-2.9.1-h8a8886c_1.tar.bz2            822 KB
libstdcxx-ng-8.2.0-hdf63c60_1.tar.bz2        2.9 MB

---------------------------------------------------
Total:   

Removing intermediate container dc76d638172b
 ---> 435492afe9be
Step 13/16 : ENV PYTHONPATH='/src/:$PYTHONPATH'
 ---> Running in 01f135b6c6a6
Removing intermediate container 01f135b6c6a6
 ---> f1c946b0c84c
Step 14/16 : WORKDIR /src
Removing intermediate container 8c2825bc25b6
 ---> bdaf24d65730
Step 15/16 : EXPOSE 8888
 ---> Running in 8c47b765468c
Removing intermediate container 8c47b765468c
 ---> fdfbc518a4f8
Step 16/16 : CMD jupyter notebook --port=8888 --ip=0.0.0.0
 ---> Running in bc0a1e194acd
Removing intermediate container bc0a1e194acd
 ---> e7d39c6caadb
Successfully built e7d39c6caadb
Successfully tagged georgedockeraccount/chestxray-no-aml-gpu:1.0.0


In [11]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu]

0f464619: Preparing 
9c4b86b2: Preparing 
4b805f25: Preparing 
d4be2f62: Preparing 
e5d9d370: Preparing 
6d4f5f2e: Preparing 
3c790502: Preparing 
ac05da10: Preparing 
19f38a85: Preparing 
d6639b96: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 


f464619: Pushing    729MB/5.488GB [14Af464619: Pushing  2.139GB/5.488GB

f464619: Pushing  2.891GB/5.488GBPushing  2.603GB/5.488GB

f464619: Pushing  4.474GB/5.488GB

f464619: Pushed   5.579GB/5.488GB1.0.0: digest: sha256:2aa5141f130c7125e092e7fcb893d80898bd76c743043f9672ddda3b16227ff2 size: 3475


In [1]:
!echo sudo nvidia-docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0 

sudo nvidia-docker run -i -t -p 10003:8888 -v /data/datadrive01/prj/tlvis:/local_dir:rw georgedockeraccount/chestxray-no-aml-gpu:1.0.0


In [14]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 382179 bytes to 00_create_docker_image.html
